In [1]:
import time
import glob as glob
from zipfile import ZipFile
from bs4 import BeautifulSoup
import dateutil.parser as dparser
from datetime import datetime
import pandas as pd
import multiprocessing as mp
import numpy as np
import re

In [2]:
# root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/downdetector' # one directory
# root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/downdetector/2017/201711' # subset of above
root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/downdetector*' # all directories

In [3]:
def extract_html_data(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    result = []
    
    timestamp = soup.find('meta', attrs={'name':'generated'})
    result.append(timestamp.get('content'))
    
    # status of the service in last 24hrs (no problems, possible problems, problems)
    try:
        status = soup.find('div', {'class' : re.compile('alert *')})
        result.append(status.get('class')[1])
    except:
        result.append('')
    
#     # time since problems started (empty unless problems are ongoing)
#     for problems_since in soup.find_all("div", {"class": "event"}):
#         try:
#             status = ' '.join(problems_since.text.split())
#             date = str(dparser.parse(status, fuzzy=True))
#             result.append([date.split()[0].split('-')[1:], date.split()[1]])
#         except:
#             # some months are apparently out of range
#             result.append('')
    
#     # TECHNICAL DEBT: problems_since is not appended if tag does not exist in the html_doc
#     if len(result) != 5:
#         result.append('')
    
#     # most reported problems at this time
#     mrp = []
#     for most_reported in soup.find_all("li"):
#         if '%' in most_reported.text:
#             mrp.append(' '.join(most_reported.text.split()))
#     result.append(mrp)

    return result

In [4]:
def read_file(file):
    source = file.split('/')[4]
    archive = ZipFile(file, 'r')
    namelist = archive.namelist()
    ret = []
    for item in namelist:
        if 'html' in item:
            html = archive.read(item)
            service = item.split('/')[1].split('.')[0]
            ts = item.split('/')[0]
            data = extract_html_data(html)
            data.insert(0, ts)
            data.insert(0, service)
            data.insert(0, source)
            ret.append(data)
    archive.close()
    return(ret)

In [5]:
files = glob.glob(root_dir + '/**/*.zip', recursive=True)
pool = mp.Pool(mp.cpu_count())

start_time = time.time()
results = pool.map(read_file, [file for file in files])
pool.close()
end_time = time.time()

print(end_time - start_time)

3072.3796997070312


In [6]:
res = []
for outer in results:
    for inner in outer:
        res.append(inner)

In [7]:
column_names = ['source','service','timestamp_dir','timestamp_site','status']
df = pd.DataFrame(res, columns=column_names)
df.head(50)

,source,service,timestamp_dir,timestamp_site,status
0,downdetector-deu-germany,spotify,20171121T190001,2017-11-21T20:00:45.027659+01:00,alert-danger
1,downdetector-deu-germany,reddit,20171121T190001,2017-11-21T20:01:17.940977+01:00,alert-success
2,downdetector-deu-germany,youtube,20171121T190001,2017-11-21T20:00:19.627327+01:00,alert-warning
3,downdetector-deu-germany,zynga,20171121T190001,2017-11-21T20:01:13.231012+01:00,alert-success
4,downdetector-deu-germany,airbnb,20171121T190001,2017-11-21T20:01:51.711265+01:00,alert-success
5,downdetector-deu-germany,amazon,20171121T190001,2017-11-21T20:00:44.680342+01:00,alert-warning
6,downdetector-deu-germany,netflix,20171121T190001,2017-11-21T20:00:41.776948+01:00,alert-warning
7,downdetector-deu-germany,slack,20171121T190001,2017-11-21T20:01:18.368305+01:00,alert-success
8,downdetector-deu-germany,snapchat,20171121T190001,2017-11-21T20:00:17.600801+01:00,alert-warning
9,downdetector-deu-germany,spotify,20171121T180001,2017-11-21T19:01:28.551709+01:00,alert-danger


In [9]:
df.to_csv(r'/home/shane/Documents/thesis/output/parsed/final/downdetector_v2.csv', index=False)